DISCLAIMER: 

This Python notebook uses python 3.12.0. Other versions may have errors.

In [23]:
%pip install fastapi uvicorn nest_asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn  

nest_asyncio.apply()

app = FastAPI()

@app.get("/")
def read_root():
    return {"Name" : "Luke"}

uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [39424]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58300 - "GET /get-data HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:58305 - "POST /predict HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39424]


Slightly more advanced 

In [ ]:
%pip install fastapi uvicorn nest_asyncio python-multipart pandas

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pandas as pd
import nest_asyncio
import uvicorn  
import numpy as np
from io import StringIO

nest_asyncio.apply()

data_records = []

app = FastAPI()

@app.on_event("startup")
def load_data():
    global data_records
    df = pd.read_csv("../MLModel/Data/Shapes.csv")
    data_records = df.to_dict(orient="records")

@app.get("/get-data")
async def gather_data():
    return JSONResponse(content=data_records)

uvicorn.run(app, host="127.0.0.1", port=8000)

C:\Users\12489\AppData\Local\Temp\ipykernel_39424\2447679922.py:15: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [39424]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60589 - "GET /get-data HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39424]


Lets go slightly further...

In [1]:
%pip install fastapi uvicorn pandas scikit-learn pydantic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import pandas as pd
import nest_asyncio
import uvicorn  
import numpy as np
from sklearn.tree import DecisionTreeClassifier

df = pd.DataFrame()

app = FastAPI()

model = None
feature_columns = ["Edges","Length","Width"]

nest_asyncio.apply()

class ShapeInput(BaseModel):
    edges: int
    length: int
    height: int

def load_data():
    global df
    df = pd.read_csv("../MLModel/Data/Shapes.csv")
    

def train_model():
    global model
    global df

    X = df[feature_columns]
    y = df["Shape"]

    model = DecisionTreeClassifier()
    model.fit(X, y)

@app.on_event("startup")
def startup():
    load_data()
    train_model()

@app.get("/get-data")
async def gather_data():
    global df
    data_records = df.to_dict(orient="records")
    return JSONResponse(content=data_records)

@app.post("/predict")
def predict_shape(input: ShapeInput):
    input_array = np.array([[input.edges, input.length, input.height]])
    prediction = model.predict(input_array)
    return {"predicted_shape": prediction[0]}

uvicorn.run(app, host="127.0.0.1", port=8000)

C:\Users\12489\AppData\Local\Temp\ipykernel_81556\3402693199.py:39: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [81556]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56703 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56704 - "GET /get-data HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [81556]
